# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.112697e+02     1.583388e+00
 * time: 0.08477997779846191
     1     9.915796e+00     9.180628e-01
 * time: 1.009108066558838
     2    -1.230176e+01     1.068952e+00
 * time: 1.138904094696045
     3    -3.449712e+01     8.475081e-01
 * time: 1.5711638927459717
     4    -4.784883e+01     6.226568e-01
 * time: 1.7188100814819336
     5    -5.718278e+01     1.855201e-01
 * time: 1.8642640113830566
     6    -5.988702e+01     1.217322e-01
 * time: 1.979780912399292
     7    -6.094240e+01     5.503898e-02
 * time: 2.0879459381103516
     8    -6.136620e+01     3.840235e-02
 * time: 2.2004098892211914
     9    -6.162073e+01     3.444705e-02
 * time: 2.3137760162353516
    10    -6.180777e+01     3.750444e-02
 * time: 2.4282209873199463
    11    -6.195621e+01     2.989796e-02
 * time: 2.5347800254821777
    12    -6.205211e+01     1.803633e-02
 * time: 2.645456075668335
    13    -6.210071e+01     1.626302e-02
 * time: 2.7577810287475

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557675
    AtomicNonlocal      14.8522653
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336818

    total               -62.261666462148
